# Блок 5_1

# Задание 1
___
С помощью какого оператора в ClickHouse можно создать представление?

Возможно несколько вариантов ответа.
- CREATE MATERIALIZED VIEW
- CREATE VIEW
- CREATE TABLE
- CREATE TABLE VIEW
- ADD VIEW

Ответ:
+ CREATE MATERIALIZED VIEW
+ CREATE VIEW

# Задание 2
___
В чём заключается различие между материализованным и обычным представлением? 

+ Обычное сохраняет запрос, который выполняется при обращении
+ Материализованное сохраняет запрос, который выполняется при обращении
+ Материализованное хранит данные на диске
+ Обычное хранит данные на диске

Ответ:
+ Обычное сохраняет запрос, который выполняется при обращении
+ Материализованное хранит данные на диске

# Задание 3
___
Данные для следующих задач: Airbnb в Берлине. Не забывайте о преобразовании столбцов к нужному типу данных!

Имеются следующие таблицы:

1) **listings** – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах; поскольку столбцов очень много, нужные перечислены в текстах самих задач

2) **calendar_summary** – информация о доступности и цене того или иного жилья по дням

+ `listing_id` – идентификатор жилья (объявления)
+ `date` – дата
+ `available` – доступность жилья в данный день (t/f)
+ `price` – цена за ночь

3) **reviews** – отзывы

+ `listing_id` –  идентификатор жилья  
+ `id` – id отзыва
+ `date` – дата отзыва
+ `reviewer_id` – id ревьюера (автора отзыва)
+ `reviewer_name` – имя автора
+ `comments` – сам отзыв

Подзапросы можно использовать в разных частях запроса, в том числе и в блоке WHERE. Потренируемся!
```
-- пример
SELECT
    column_1,
    column_2
FROM
    table
WHERE
    column_2 = (SELECT MAX(column_2) 
                FROM table)
```
Сначала оставьте только те объявления, где оценка на основе отзывов выше среднего, а число отзывов в месяц составляет строго меньше трёх. Затем отсортируйте по убыванию две колонки: сначала по числу отзывов в месяц, потом по оценке. В качестве ответа укажите id объявления из первой строки.

+ `review_scores_rating` – оценка на основе отзывов
+ `reviews_per_month` – число отзывов в месяц
+ `id` – идентификатор объявления

Таблица – `listings`. Наивысшие показатели = сортировка по убыванию.

Правильный запрос:
```
SELECT 
    id, 
    reviews_per_month,
    toFloat64OrNull(review_scores_rating) as review_scores_rating
FROM listings
WHERE review_scores_rating > (
                            SELECT 
                                AVG(toFloat64OrNull(review_scores_rating))
                            FROM listings
                            )
    AND reviews_per_month < 3
ORDER BY
    review_scores_rating DESC,
    reviews_per_m
```
Ответ:
```
22476940
```

# Задание 4
___
Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем. 

+ `id` – идентификатор объявления
+ `host_id` – идентификатор хозяина
+ `room_type` – тип жилья ('Private room')
+ `latitude` – широта
+ `longitude` – долгота
+ 52.5200 с.ш., 13.4050 в.д – координаты центра Берлина

В качестве ответа укажите идентификатор хозяина (`host_id`), сдающего данную комнату.

Правильный запрос:
```
SELECT
    host_id,
    room_type,
    longitude,
    latitude,
    geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) as dist
FROM
    listings
WHERE 
    dist < (SELECT 
                AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude))) as AvgDist
            FROM
                listings
            WHERE 
                room_type = 'Private room'
            )
    AND room_type = 'Private room'
ORDER BY
    dist DESC
LIMIT 10
```
Ответ:
```
163065208
```

# Задание 5
___
Представим, что вы планируете снять жилье в Берлине на 7 дней, используя более хитрые фильтры, чем предлагаются на сайте.

В этой задаче можно потренироваться в написании подзапросов, но задание можно решить и без них.

Отберите объявления из таблицы `listings`, которые:

+ находятся на расстоянии от центра меньше среднего (здесь нам пригодится запрос из предыдущего задания)
+ обойдутся дешевле `100$` в день (`price` с учетом `cleaning_fee`, который добавляется к общей сумме за неделю, т.е его нужно делить на кол-во дней)
+ имеют последние отзывы (`last_review`), начиная с 1 сентября 2018 года
+ имеют WiFi в списке удобств (`amenities`)

Отсортируйте полученные значения по убыванию `review_scores_rating` (не забудьте перевести строку к численному виду) и в качестве ответа укажите `host_id` из первой строки. 

Правильный запрос:
```
SELECT
    host_id,
    review_scores_rating,
    geoDistance(toFloat64OrNull(longitude), toFloat64OrNull(latitude), 13.4050, 52.5200) as geo_distance,
    last_review,
    amenities
FROM 
    listings
WHERE 
    geo_distance <
    (SELECT AVG(geoDistance(toFloat64OrNull(longitude), toFloat64OrNull(latitude), 13.4050, 52.5200)) FROM listings)
AND
    (toFloat32OrNull(replaceRegexpAll(price, '[$,]', '')) + toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', '')) / 7) < 100
AND
    last_review >='2018-09-01'
AND
    multiSearchAnyCaseInsensitive(amenities, ['wifi'])!=0
ORDER BY 
    review_scores_rating DESC
LIMIT 10
```
Ответ:
```
39740677
```

# Задание 6
___
Давайте найдем в таблице `calendar_summary` те доступные (`available='t'`) объявления, у которых число отзывов от уникальных пользователей в таблице `reviews` выше среднего.

Для этого с помощью конструкции `WITH` посчитайте среднее число уникальных `reviewer_id` из таблицы `reviews` на каждое жильё, потом проведите джойн таблиц `calendar_summary` и `reviews` по полю `listing_id`(при этом из таблицы calendar_summary должны быть отобраны уникальные `listing_id`, отфильтрованные по правилу `available='t'`). Результат отфильтруйте так, чтобы остались только записи, у которых число отзывов от уникальных людей выше среднего.

Отсортируйте результат по возрастанию `listing_id` и в качестве ответа впишите количество отзывов от уникальных пользователей из первой строки.

Для решения проблем со вложенными агрегационными функциями (и агрегационными функциями там, где их не ждут) как раз очень помогут подзапросы.

Правильный запрос:
```
WITH
(SELECT AVG(review_num) 
FROM 
    (SELECT 
        COUNT(DISTINCT(reviewer_id)) review_num, 
        listing_id  
     FROM 
         reviews 
     GROUP BY 
         listing_id)) AS avg_review

SELECT
listing_id,
review_num
FROM (
	SELECT
    	COUNT(DISTINCT(reviewer_id)) review_num,
    	listing_id
	FROM reviews
	GROUP BY listing_id
	) AS r
JOIN (
	SELECT
    	listing_id
	FROM calendar_summary
	WHERE available='t' 
GROUP BY listing_id
	) AS c
ON
	c.listing_id = r.listing_id
WHERE review_num > avg_review  
ORDER BY listing_id
LIMIT 10
```
Ответ:
```
118
```

# Задание 7
___
Возвращаемся к данными мобильного приложения!

**installs** — содержит данные об установках приложения по дням.

+ `DeviceID` — идентификатор устройства, на которое было установлено приложение;
+ `InstallationDate` — дата установки приложения;
+ `InstallCost` — цена установки приложения в рублях;
+ `Platform` — платформа, на которой было установлено приложение (iOS/ Android);
+ `Source` — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).

**events** — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.

+ `DeviceID` — идентификатор устройства, на котором используется приложение;
+ `AppPlatform` — платформа, на которой используется приложение (iOS/ Android);
+ `EventDate` — дата, за которую собрана статистика;
+ `events` — количество просмотров всех товаров за этот день у этого DeviceID.

**checks** — содержит данные о покупках пользователей в приложении по дням

+ `UserID` — идентификатор пользователя;
+ `Rub` — суммарный чек пользователя на дату;
+ `BuyDate` — дата, за которую собрана статистика.

**devices** – чтобы просмотры и установки можно было объединить с покупками

+ `DeviceID` — идентификатор устройства;
+ `UserID` — идентификатор пользователя.

С помощью оператора `CASE` в SQL можно составлять условные конструкции, выполнять проверку условий и возвращать результат в зависимости от их выполнения. Самих условий под оператором `WHEN` может быть сколько угодно, конструкцию необходимо завершить оператором `END`. Структура выглядит следующим образом:
```
SELECT 
    CASE
        -- если значение в column_1 больше, чем в column_2, в новый столбец будет записана единичка
        WHEN column_1 > column_2 THEN 1 
        -- если значение в column_1 меньше, чем в column_2, в новый столбец будет записана двойка
        WHEN column_1 < column_2 THEN 2
        -- во всех остальных случаях – тройка
        ELSE 3
    END AS cond_1 -- END – завершаем конструкцию, а новый столбец называем cond_1
FROM
    table
```
Какие из перечисленных условных конструкций составлены некорректно?
+ WHERE price > 1000 THEN 'ok' ELSE 'not ok' END
+ CASE WHEN price > 1000 THEN 'ok' ELSE 'not ok'
+ CASE WHEN price>1000 THEN 'ok' WHEN price<500 THEN 'not ok' ELSE 'normal' END

Ответ:
+ WHERE price > 1000 THEN 'ok' ELSE 'not ok' END
+ CASE WHEN price > 1000 THEN 'ok' ELSE 'not ok'

# Задание 8
___
Теперь задача! Используйте таблицу `checks` и разделите всех покупателей на сегменты:

Правые границы берутся не включительно, например, чек в 10 рублей будет относиться к сегменту С

+ А — средний чек покупателя менее 5 ₽
+ B — средний чек покупателя от 5-10 ₽
+ C — средний чек покупателя от 10-20 ₽
+ D — средний чек покупателя от 20 ₽

Отсортируйте результирующую таблицу по возрастанию `UserID `и укажите сегмент четвертого пользователя.

Правильный запрос:
```
SELECT
    UserID,
    CASE
        WHEN avg(Rub) < 5 THEN 'A'
        WHEN avg(Rub) >= 5 AND avg(Rub) < 10 THEN 'B'
        WHEN avg(Rub) >= 10 AND avg(Rub) < 20 THEN 'C'
        ELSE 'D'
    END AS segment
FROM checks
GROUP BY UserID
ORDER BY UserID ASC
LIMIT 10
```
Ответ:
```
C
```

# Задание 9
___
Используйте предыдущий запрос как подзапрос и посчитайте, сколько клиентов приходится на каждый сегмент и сколько доходов он приносит. Отсортируйте результат по убыванию суммы доходов на сегмент и в качестве ответа укажите наибольшую сумму.

Правильный запрос:
```
SELECT
    segment,
    COUNT(DISTINCT UserID),
    sum(Rub) as Revenue
FROM
    checks
JOIN
    (
    SELECT
    UserID,
    CASE
        WHEN avg(Rub) < 5 THEN 'A'
        WHEN avg(Rub) >= 5 AND avg(Rub) < 10 THEN 'B'
        WHEN avg(Rub) >= 10 AND avg(Rub) < 20 THEN 'C'
        ELSE 'D'
    END AS segment
    FROM checks
    GROUP BY UserID
    ) as sub
USING (UserID)
GROUP BY segment
ORDER BY Revenue DESC
LIMIT 10
```
Ответ:
```
250551286
```

# Задание 10
___
Вернемся к таблице AirBnb. Предположим, что в выборе жилья нас интересует только два параметра: наличие кухни (`kitchen`) и гибкой системы отмены (`flexible`), причем первый в приоритете.

Создайте с помощью оператора `CASE` колонку с обозначением группы, в которую попадает жилье из таблицы `listings`:

1) 'good', если в удобствах (`amenities`) присутствует кухня и система отмены (`cancellation_policy`) гибкая
2) 'ok', если в удобствах есть кухня, но система отмены не гибкая
3) 'not ok' во всех остальных случаях

Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк, в качестве ответа укажите `host_id` первой строки.

Обратите внимание, что `cancellation_policy` - это отдельная колонка, по ней необходимо смотреть систему отмены

Правильный запрос:
```
SELECT 
    host_id,
CASE
    WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen'])!=0 AND cancellation_policy = 'flexible' THEN 'good'
    WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen'])!=0 AND cancellation_policy != 'flexible' THEN 'ok'
ELSE 'not ok'
END my_group
FROM 
    listings 
ORDER BY 
    my_group LIMIT 5
```
Ответ:
```
27607502
```

# Блок 5_2

# Задание 1
___
Сопоставьте операторы с соответствующим описанием:

Ответ:
+ ADD COLUMN - добавить столбец
+ DELETE WHERE - удалить строки, где выполняется условие
+ DROP COLUMN - удалить столбец
+ DROP TABLE - удалить таблицу
+ INSERT INTO - записать данные в таблицу

# Задание 2
___
Напишите запрос для создания таблицы со следующими параметрами, также подобрав подходящий тип данных.

Название таблицы: `reviews`
База данных: `test`
Столбцы:

- `listing_id` – идентификатор объявления, может быть только положительным и целым числом, 32-битный тип данных

- `id` – идентификатор хозяина, может быть только положительным и целым числом, 32-битный тип данных

- `date` – дата со временем (2020-01-01 00:00:00), часовой пояс – 'Europe/Moscow'

- `reviewer_id` – идентификатор ревьюера, может быть только положительным и целым числом, 32-битный тип данных

- `reviewer_name` – имя того, кто оставил отзыв

- `comments` - текст отзыва


+ Движок: `MergeTree`
+ Сортировка: `listing_id`, `id`

Правильные запрос:
```
CREATE TABLE test.reviews (
    listing_id UInt32,
    id UInt32,
    date DateTime('Europe/Moscow'),
    reviewer_id UInt32,
    reviewer_name String,
    comments String
    ) 

ORDER BY (listing_id, id)
ENGINE = MergeTree
```

# Задание 3
___
К вам пришел коллега с новостями: оказывается, в поле `date` будет записываться только дата, без времени, поэтому нужно изменить тип данных!

Напишите запрос, который модифицирует тип данных, и введите его в поле ниже без кавычек и лишних пробелов. Не забудьте указать базу данных `test` перед названием таблицы! 

Правильные запрос:
```
ALTER TABLE test.reviews MODIFY COLUMN date Date
```

# Задание 4
___
Предположим, ваш коллега вставил данные, но что-то перепутал. Часть строк с комментариями осталась совершенно пустой! 

Напишите запрос, который удалит из таблицы `test.reviews` те строки, где в столбце `comments` встречаются пустые значения (`''`). Введите его в поле ниже без кавычек и лишних пробелов.

Правильные запрос:
```
ALTER TABLE test.reviews DELETE WHERE comments = ''
```

# Задание 5
___
С помощью какого запроса можно создать обычное представление над таблицей `test.reviews`, которое будет содержать все записи из `test.reviews`, сгруппированные по `reviewer_id` с подсчитанным количеством отзывов (`id`) на каждого пользователя?

Варианты запросов:
+ ALTER TABLE test.reviews_number UPDATE id = COUNT(id)
+ CREATE VIEW test.reviews_number AS (SELECT reviewer_id, COUNT(id) reviews_count FROM test.reviews GROUP BY reviewer_id)
+ CREATE VIEW test.reviews_number AS (SELECT reviewer_id, SUM(id) reviews_count GROUP BY reviewer_id)
+ CREATE MATERIALIZED VIEW test.reviews_number AS (SELECT reviewer_id, COUNT(id) reviews_count GROUP BY reviewer_id)
+ CREATE TABLE IF NOT EXISTS test.reviews_number AS (SELECT reviewer_id, COUNT(id) reviews_count GROUP BY reviewer_id)

Правильные запрос:
```
CREATE VIEW test.reviews_number AS (SELECT reviewer_id, COUNT(id) reviews_count FROM test.reviews GROUP BY reviewer_id)

```

# Задание 6
___
С помощью какого запроса можно создать новый столбец `reviewer_score` в таблице `reviews` после столбца `reviewer_name`?

Варинаты запросов:
+ ALTER TABLE test.reviews AFTER reviewer_name ADD COLUMN reviewer_score UInt8
+ ALTER TABLE test.reviews ADD COLUMN reviewer_score UInt8
+ ALTER TABLE test.reviews INSERT COLUMN reviewer_score UInt8 AFTER reviewer_name
+ ALTER TABLE test.reviews ADD COLUMN reviewer_score UInt8 AFTER reviewer_name
+ ALTER TABLE test.reviews AFTER reviewer_name INSERT reviewer_score UInt8

Правильные запрос:
```
ALTER TABLE test.reviews ADD COLUMN reviewer_score UInt8 AFTER reviewer_name 
```

# Задание 7
___
Напишите запрос для добавления в таблицу `test.reviews` колонки `price` после колонки `comments`, которая может быть числом с плавающей точкой, 32-битный тип данных.

Правильные запрос:
```
ALTER TABLE test.reviews ADD COLUMN price Float32 AFTER comments
```

# Задание 8
___
Напишите запрос, который удвоит `price` для всех строк с датой (`date`) после 2019-01-01.

Обратите внимание, что для сравнения с датой в формате '2019-01-01' можно не преобразовывать данные в колонке `date`.

Правильные запрос:
```
ALTER TABLE test.reviews UPDATE price=price*2 WHERE date>'2019-01-01'
```